In [143]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import os
import google.generativeai as genai
import textwrap
from IPython.display import display
from IPython.display import Markdown

In [144]:
GOOGLE_API_KEY=""

genai.configure(api_key=GOOGLE_API_KEY)

In [145]:
model = genai.GenerativeModel('gemini-pro')

In [146]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model2 = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model2, tokenizer=tokenizer,grouped_entities=True)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [147]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
def connect_to_postgres():
    with open('database.yaml', 'r') as yaml_file:
        db_config = yaml.safe_load(yaml_file)
    try:
        connection = psycopg2.connect(
            user=db_config['database']['user'],
            password=db_config['database']['password'],
            host=db_config['database']['host'],
            port=db_config['database']['port'],
            database=db_config['database']['name']
        )
        print("Connection to PostgreSQL database successful")
        return connection
    except Error as e:
        print(f"Error while connecting to PostgreSQL: {e}")

In [148]:
response = model.generate_content("List competitors of Bajaj Auto")

In [149]:
to_markdown(response.text)

> * **Hero MotoCorp**
> * **Honda Motorcycle & Scooter India**
> * **TVS Motor Company**
> * **Yamaha Motor India**
> * **Suzuki Motorcycle India**
> * **Royal Enfield**
> * **KTM**
> * **BMW Motorrad**
> * **Harley-Davidson**
> * **Triumph Motorcycles**
> * **Kawasaki**
> * **Hyosung**
> * **Benelli**
> * **Keeway**

In [150]:
competitors=[]

for i in nlp(response.text):
    competitors.append(i["word"])

In [151]:
for i in competitors:
    if(i[0]=='#'):
        i = re.sub('^#+', '', i)
        temp=competitors[-1]
        del competitors[-1]
        temp+=i
        if temp not in competitors:
            competitors.append(temp)
        continue
        
    if i not in competitors:
        competitors.append(i)

In [152]:
competitors

['Hero MotoCorp',
 'Honda Motorcycle & Scooter India',
 'TVS Motor Company',
 'Yamaha Motor India',
 'Suzuki Motorcycle India',
 'Royal Enfield',
 'KTM',
 'BMW Motorrad',
 'Harley',
 'Davidson',
 'Triumph Motorcycles',
 'Kawasaki',
 'Ben']

In [153]:
i=competitors[0]

response = model.generate_content("Give insights and compare Bajaj Auto and "+i)

to_markdown(response.text)

> **Bajaj Auto vs. Hero MotoCorp**
> 
> **Overview**
> 
> * **Bajaj Auto:** Indian multinational automotive manufacturing company based in Pune. Known for its affordable motorcycles and three-wheelers.
> * **Hero MotoCorp:** Indian multinational motorcycle manufacturer based in Gurgaon. World's largest two-wheeler manufacturer by volume.
> 
> **Market Position**
> 
> * **Dominance:** Hero MotoCorp is the dominant player in India's motorcycle market, with a market share of around 40%. Bajaj Auto has a smaller market share of approximately 12%.
> * **Export Focus:** Bajaj Auto has a strong export focus, exporting motorcycles to over 70 countries. Hero MotoCorp has recently been expanding its export operations.
> 
> **Product Portfolio**
> 
> * **Major Products:** Both companies primarily focus on producing motorcycles.
> * **Range:** Bajaj Auto offers a wider range of motorcycles, from entry-level models to high-performance bikes. Hero MotoCorp has a more focused portfolio, with a strong presence in the commuter and economy segments.
> * **Innovation:** Bajaj Auto is known for its innovative designs and technologies, such as its patented DTS-i engine. Hero MotoCorp emphasizes reliability and fuel efficiency in its products.
> 
> **Financial Performance**
> 
> * **Revenue:** Hero MotoCorp consistently generates higher revenue than Bajaj Auto due to its larger market share.
> * **Profitability:** Bajaj Auto has typically maintained higher profit margins than Hero MotoCorp, reflecting its focus on higher-margin products and export markets.
> * **Growth:** Both companies have experienced steady growth in recent years, with Hero MotoCorp leading the pack in terms of market share.
> 
> **Strengths and Weaknesses**
> 
> **Bajaj Auto:**
> 
> * **Strengths:** Export focus, innovative designs, higher profit margins.
> * **Weaknesses:** Relatively smaller market share in India.
> 
> **Hero MotoCorp:**
> 
> * **Strengths:** Dominant market position in India, strong brand recognition, focus on fuel efficiency.
> * **Weaknesses:** Dependence on domestic market, limited product diversification.
> 
> **Competitive Landscape**
> 
> The Indian two-wheeler market is highly competitive, with several other players such as Honda Motorcycles and Scooters India (HMSI) and TVS Motor Company. However, Bajaj Auto and Hero MotoCorp remain the two largest players, controlling over 50% of the market share.
> 
> **Conclusion**
> 
> Bajaj Auto and Hero MotoCorp are both leading players in the Indian two-wheeler industry. While Hero MotoCorp has a dominant market position, Bajaj Auto stands out with its export focus and innovative product portfolio. The competition between these two companies is expected to remain intense in the future, with both vying for market share and growth opportunities.